## 에이전트에 미들웨어 추가하기

에이전트에 미들웨어를 추가하는 방법을 알아봅니다. 미들웨어를 사용하면 로깅, 보안 및 기타 공통적인 관심사를 위해 에이전트 상호 작용을 가로채고 수정할 수 있습니다.

### 미들웨어 작성하기

간단한 미들웨어를 작성해서 에이전트의 동작을 확장할 수 있습니다. 예를 들어, 에이전트가 응답을 생성하기 전에 로그를 기록하는 미들웨어를 추가할 수 있습니다.

In [6]:
from agent_framework import AgentRunContext
from typing import Callable, Awaitable

async def logging_agent_middleware(
    context: AgentRunContext,
    next: Callable[[AgentRunContext], Awaitable[None]],
) -> None:
    """Simple middleware that logs agent execution."""
    print("Agent starting...")

    # Continue to agent execution
    await next(context)

    print("Agent finished!")

### Agent를 생성하고, 미들웨어 추가하기

In [ ]:
from agent_framework.azure import AzureOpenAIChatClient
from azure.identity import AzureCliCredential

agent = AzureOpenAIChatClient(credential=AzureCliCredential()).as_agent(
    name="GreetingAgent",
    instructions="You are a friendly greeting assistant who responds in korean.",
    middleware=[logging_agent_middleware]   # Add your middleware here
)

async def main():
    result = await agent.run("Hello!")
    print(result.text)

await main()

Agent starting...
Agent finished!
안녕하세요! 만나서 반가워요. 무엇을 도와드릴까요?


### 미들웨어와 Function 도구를 사용하는 에이전트 생성

에이전트가 Function 도구를 사용하는 경우, Function 호출도 수신할 수 있습니다.

In [11]:
from agent_framework import FunctionInvocationContext

def get_time():
    """Get the current time."""
    from datetime import datetime
    return datetime.now().strftime("%H:%M:%S")

async def logging_function_middleware(
    context: FunctionInvocationContext,
    next: Callable[[FunctionInvocationContext], Awaitable[None]],
) -> None:
    """Middleware that logs function calls."""
    print(f"호출한 function : {context.function.name}")

    await next(context)

    print(f"function 결과: {context.result}")

# Add both the function and middleware to your agent
agent = AzureOpenAIChatClient(credential=AzureCliCredential()).as_agent(
    name="TimeAgent",
    instructions="You can tell the current time.",
    tools=[get_time],
    middleware=[logging_function_middleware],
) 
async def main():
    result = await agent.run("What time is it?")
    print(result.text)

await main()

호출한 function : get_time
function 결과: 22:16:14
The current time is 22:16 (10:16 PM).
